# Spark
  - Processa dados em memória
  - Opera em Cluster
  - Particionamento (divisão de dados)
  - Paralelismo
  - Redundância

# Particionamento
  - Por padrão dados são particionados de acordo com o número de núcleos
  - Cada partição fica em um nó e tem uma task
  - Dados são particionados por padrão e dependem de vários fatores e configurações
  - Podemos particionar explicitament em disco (partitionBy)
  - Ou em memória repartition() ou coalesce()
  - Spark Shuffle 
    - É um mecanismo para redistribuir ou  é um mecanismo para redistribuir ou re-particionar dados para que os dados sejam agrupados de forma diferente nas partições. O Spark shuffle é uma operação muito cara, pois move os dados entre executores ou mesmo entre nós do trabalhador em um cluster.
  - Bucketing
    - Semelhante a particionamento, porém com número fixo de partições
    - Ideal para coluna com alta cardinalidade
    - Pode ser usado com conjunto Particionamento

In [ ]:
%%sh
pip install spark
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=26ab3a4bc8777549de8202967432864243d8cabd8969fe814c977d82ca901865
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e04c4a73cdab2578d65936eb676605c4620c5db2f22c3d38798be54f88f9e664
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('spark_particionamento').getOrCreate()

In [ ]:
# Criando o database desp
spark.sql('CREATE DATABASE desp').show()

++
||
++
++



In [ ]:
# entrando no banco desp
spark.sql('USE desp').show()

++
||
++
++



In [ ]:
churn = spark.read.csv('/content/Churn.csv', header=True, inferSchema=True, sep=';')
churn.show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       

In [ ]:
# Cria uma tabela fisica no nosso banco de dados desp
# Cria um particionamento atráves da coluna Geography
churn.write.partitionBy('Geography').saveAsTable('churn_geo')

In [ ]:
# consultando a tabela churn_geo
# na pasta desp.db/churn_geo
# foi criado 3 pastas para cada geography
# https://prnt.sc/ZvvEXlx4KfeM
spark.sql('SELECT * FROM churn_geo').show(5)

+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|CreditScore|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|Geography|
+-----------+------+---+------+--------+-------------+---------+--------------+---------------+------+---------+
|        619|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|   France|
|        502|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|   France|
|        699|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|   France|
|        822|  Male| 50|     7|       0|            2|        1|             1|         100628|     0|   France|
|        501|  Male| 44|     4|14205107|            2|        0|             1|         749405|     0|   France|
+-----------+------+---+------+--------+-------------+---------+--------------+---------------+-

# Criando um Bucket

In [ ]:
# Bucket é usado quando a cardinalidade é muito alta
# bucketBy: 1º param: número de partições; 2º tabela
# estrutura churn_geo_2: https://prnt.sc/Ae-BYQH1emnR
# apesar de termos definido 3 partições, ele criou apenas 2, porque o número de dados é pequeno
churn.write.bucketBy(3, 'Geography').saveAsTable('churn_geo_2')

In [ ]:
spark.sql('SELECT * FROM churn_geo_2').show(5)

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+-------

# Cache e Persist
 - Cache - padrão em memória, ou seja, coloca o objeto em memória
 - Persist - definido pelo usuário

# StorageLevel
 
 
 - DISK_ONLY
 - DISK_ONLY_2
 - DISK_ONLY_3
 - MEMORY_AND_DISK: Padrão para DataFrame. Armazena as partiçõesq ue não    cabem em memória em disco
 - MEMORY_AND_DISK_2
 - MEMORY_AND_DISK_SER
 - MEMORY_AND_DISK_SER_2
 - MEMORY_ONLY: Padrão para RDD, porém se não caber na memória será reprocessado a cada consulta
 - MEMORY_ONLY_2
 - MEMORY_ONLY_SER
 - MEMORY_ONLY_SER_2
 - OFF_HEAP

In [ ]:
from pyspark import StorageLevel

In [ ]:
spark.sql('USE desp').show()

++
||
++
++



In [ ]:
spark.sql('SHOW TABLES').show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|     desp|   churn_geo|      false|
|     desp|  churn_geo2|      false|
|     desp| churn_geo_2|      false|
|     desp|despachantes|      false|
+---------+------------+-----------+



In [ ]:
# Ler a tabela despachantes e transformar em um df
df = spark.sql('SELECT * FROM despachantes')
df.show(5)

+---+-------------------+-----+-------------+---+-------------------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|         2020-08-11|
+---+-------------------+-----+-------------+---+-------------------+
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05 00:00:00|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05 00:00:00|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05 00:00:00|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05 00:00:00|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05 00:00:00|
+---+-------------------+-----+-------------+---+-------------------+
only showing top 5 rows



In [ ]:
# verificar o storageLevel

# 1º False: Não está em disco
# 2º False: Não está em memória
# 3º False: Não está em OFFHEAP
# 4º False: Não está serializado
# 5º False: Quantidade de repliacao
df.storageLevel

StorageLevel(True, True, False, True, 1)

In [ ]:
# Cria o cache para o df
df.cache()

DataFrame[1: int, Carminda Pestana: string, Ativo: string, Santa Maria: string, 23: int, 2020-08-11: timestamp]

In [ ]:
# consultando novamente o storageLevel

# 1º False: Está em disco
# 2º False: Está em memória
# 3º False: Não está em OFFHEAP
# 4º False: Eestá serializado
# 5º False: Quantidade de repliacao
df.storageLevel

StorageLevel(True, True, False, True, 1)

In [ ]:
df.persist(StorageLevel.DISK_ONLY)

DataFrame[1: int, Carminda Pestana: string, Ativo: string, Santa Maria: string, 23: int, 2020-08-11: timestamp]

In [ ]:
# Remove o dado em cache
df.unpersist()

DataFrame[1: int, Carminda Pestana: string, Ativo: string, Santa Maria: string, 23: int, 2020-08-11: timestamp]

In [ ]:
df.persist(StorageLevel.DISK_ONLY)

DataFrame[1: int, Carminda Pestana: string, Ativo: string, Santa Maria: string, 23: int, 2020-08-11: timestamp]

In [ ]:
df.storageLevel

StorageLevel(True, False, False, False, 1)